# 패키지 불러오기

In [1]:
!openai --version

openai 1.35.7


In [2]:
!pwd

/Users/heungrokoh/python_workspace/work/voucher_paprika


In [10]:
import os
import time
import yaml
import re
import json
import requests
import pickle

from api_utils import *

from openai import OpenAI
import pandas as pd
import numpy as np
import ipywidgets as widgets # interactive display
from tqdm import tqdm # progress bar
from dotenv import load_dotenv
from collections import defaultdict

# ENV, Config 파일 읽기

In [12]:
# .env 파일에서 환경 변수 로드(API 키)
load_dotenv()

True

In [13]:
# YAML 파일 열기
yaml_path = 'config.yaml' # todo: config 파일과 합치기

with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

# gpt prompt 설정
sentiment_instructions = config['gptapi']['sentiment']['instructions']
sentiment_few_shots = config['gptapi']['sentiment']['few_shot_examples']
theme_instructions = config['gptapi']['theme']['instructions']
theme_few_shots = config['gptapi']['theme']['few_shot_examples']


# 함수 정의(todo: 모듈화)

In [ ]:
# 모듈

def get_response(messages):
    """
    model: 모델 종류
    messages: 사용자의 입력과 모델의 출력을 교환하는 메시지 목록
    max_tokens: 생성될 응답의 최대 길이
    temperature: 생성될 응답의 다양성(0.0 ~ 1.0) 0.0은 가장 확실한 답변을, 1.0은 가장 다양한 답변을 생성
    stream: 응답을 한 번에 반환할지 여부. False로 설정하면 한 번에 반환
    """
    try:
        # client: OpenAI API 인스턴스
        response = client.chat.completions.create(
            model = "gpt-4o",
            messages = messages,
            # max_tokens = 150,
            temperature = 0.5,
            stream = False)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# JSON 응답에서 raw_text 항목을 추출하는 함수
def extract_raw_text(content):
    return content.get('raw_text', '')

# JSON 응답에서 issue 항목을 추출하는 함수
def extract_issues(content):
    return content.get('issue', [])

# JSON 응답에서 각 issue에 대한 sentiment 값을 추출하는 함수
def extract_sentiments(content):
    return content.get('sentiment', {})

# JSON 응답에서 sentiment_all 값을 추출하는 함수
def extract_sentiment_all(content):
    return content.get('sentiment_all', '')

# 문자열을 딕셔너리로 변환하는 함수
def convert_to_dict(content):
    try:
        return json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    
# JSON 데이터를 변환하고 추출된 데이터를 DataFrame으로 변환하는 함수
def convert_to_dataframe(content_list):
    data = []
    
    for content in content_list:
        if not content:
            continue  # content가 None이거나 비어 있을 때 건너뜀
        
        content_dict = convert_to_dict(content)
        if not content_dict:
            continue  # JSON 변환에 실패한 경우 건너뜀
        
        raw_text = extract_raw_text(content_dict)
        issues = extract_issues(content_dict)
        sentiments = extract_sentiments(content_dict)
        sentiment_all = extract_sentiment_all(content_dict)
        
        # 각 issue와 sentiment를 별도로 저장
        for issue, sentiment in sentiments.items():
            data.append({
                "raw_text": raw_text,
                "issue": issue,
                "sentiment": sentiment,
                "sentiment_all": sentiment_all
            })
    
    # 데이터가 있을 경우에만 DataFrame 생성
    if data:
        return pd.DataFrame(data)
    else:
        print("No valid data to create DataFrame.")
        return pd.DataFrame()  # 빈 DataFrame 반환
    
def map_issues_to_themes(issues_list, clustering_response):
    themes = []
    for issues in issues_list:
        issue_themes = set()  # 중복을 방지하기 위해 set 사용
        for issue in issues:
            for theme, issue_group in clustering_response.items():
                if issue in issue_group:
                    issue_themes.add(theme)
        themes.append(", ".join(issue_themes))  # 여러 테마가 있을 경우 ","로 결합
    return themes

# 후처리 함수 정의(리스트, 딕셔너리 형태를 문자열로 변환)
def clean_format(value):
    if isinstance(value, list):
        return ', '.join(value)  # 리스트의 경우 ,로 구분하여 문자열로 변환
    if isinstance(value, dict):
        return ', '.join([f"{k}: {v}" for k, v in value.items()])  # 딕셔너리의 경우 key: value 형식으로 변환
    return value  # 이미 문자열인 경우는 그대로 반환

def classify_issues(issues):
    # 'instructions'는 모델의 행동 방침을 설정하므로 system 메시지에 포함
    messages = [
        {"role": "system", "content": theme_instructions},
        {"role": "user", "content": f"다음 이슈들을 의미적으로 비슷한 그룹으로 나누고, 각 그룹의 공통 테마를 도출해줘: {issues}"},
        {"role": "user", "content": f"예시: {theme_few_shots}"}
    ]
    
    return get_response(messages)

# themes 리스트의 각 JSON 문자열을 dict 형태로 변환하여 병합하는 함수
def merge_themes(themes):
    merged_dict = defaultdict(list)  # 리스트를 기본값으로 갖는 defaultdict 생성

    for theme in themes:
        # JSON 문자열을 Python dict로 변환
        theme_dict = json.loads(theme)
        
        # 각 key와 value를 병합
        for key, values in theme_dict.items():
            for value in values:
                if value not in merged_dict[key]:  # 중복된 value가 없을 경우만 추가
                    merged_dict[key].append(value)
    
    return dict(merged_dict)  # defaultdict을 일반 dict로 변환하여 반환

# 'theme' 컬럼을 생성하는 함수(이슈를 테마로 매핑)
def find_theme(merged_themes, issue):
    for theme, issues in merged_themes.items():
        if issue in issues:
            return theme
    return '기타'  # 해당하는 theme이 없을 경우 '기타'로 표시

# 인스턴스 생성

In [14]:
# OpenAI API 클라이언트 생성
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 파일 읽기

In [15]:
# 처리할 데이터
df = pd.read_csv('data/제공/제공_네이버쇼핑리뷰.csv')
df.head()

,내용,구매처,평점,작성날짜,사용자,상품
0,조아용 조아용 조아용 ㅎㅎ,오아시스486,5,24.05.15.,rorn****,스탠바이미
1,너무 만족하며 사용중입니다,Ader,5,24.05.15.,arde****,스탠바이미
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,Ader,5,24.05.15.,jeon****,스탠바이미
3,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,Ader,5,24.05.15.,pjw9****,스탠바이미
4,살까말까 했는데 좋아요,11번가,4,24.05.15.,m0******,스탠바이미


# 적용 영역

## Step 1) gpt api 호출

In [16]:
text_apply = df['내용']

In [17]:
print(sentiment_instructions)

넌 글을 요약하는 전문가야. 어떤 제품에 대한 리뷰 원문(raw_text)을 주면 다음 지시사항에 따라 예시와 같이 결과를 생성해줘.
<지시사항>
1. issue는 원문의 주요 사항을 요약한 것입니다. 동일하거나 유사한 표현은 하나의 issue로 통합합니다. 예: “좋은 화질”, “뛰어난 화질” → “우수한 화질”, “가정 및 캠핑에서 사용” → “캠핑 및 가정에서 사용”.
2. 각 issue에 대해 감정 분석을 수행하고 sentiment로 표현합니다.
3. sentiment는 긍정, 부정, 중립 중 하나로 표시하며, 명확한 감정 평가가 어려운 경우 중립으로 처리합니다.
4. 응답은 순수 JSON 형식으로만 제공하며, 코드블록이나 마크다운 포맷은 사용하지 않습니다.
</지시사항>



In [18]:
print(sentiment_few_shots)

<예시>
{
  "raw_text": "자취하는데 뭔 티비야 .. 하면서 노트북이나 패드로 유튜브, ott 시청했었는데 확실히 더 큰 화면으로 보니까 너무 조아요설거지하거나 가구 배치 바꿀 때도 쉽게 움직여서 이동시킬 수 있으니까 정말 좋습니다..삶의 질이 향상됐어요 ㅎㅎ",
  "issue": [
    "더 큰 화면으로 OTT 시청 좋음",
    "이동 편리"
  ],
  "sentiment": {
    "더 큰 화면으로 OTT 시청 좋음": "긍정",
    "이동 편리": "긍정"
  },
  "sentiment_all": "긍정"
},
{
  "raw_text": "핸드폰으로 ott를 보다보니 항상 아쉬웠는데 고민고민하다질렀어요~^^ 주문후 다음날 오전에 배송설치했고 생각한대로 디자인이나 품질 맘에들어요",
  "issue": [
    "OTT 시청",
    "빠른 배송",
    "디자인 만족",
    "품질 만족"
  ],
  "sentiment": {
    "OTT 시청": "중립",
    "빠른 배송": "긍정",
    "디자인 만족": "긍정",
    "품질 만족": "긍정"
  },
  "sentiment_all": "긍정"
},
{
  "raw_text": "너무 좋아요무겁지만 바퀴가 있어 이동이 용이 하고사운드,화질등 나무랄게 없어요",
  "issue": [
    "무겁지만 이동 용이",
    "우수한 사운드",
    "우수한 화질"
  ],
  "sentiment": {
    "무겁지만 이동 용이": "중립",
    "우수한 사운드": "긍정",
    "우수한 화질": "긍정"
  },
  "sentiment_all": "긍정"
},
{
  "raw_text": "밥 먹을때 자기전에 이동하면서 보고있는데 너무 편하고 좋아요",
  "issue": [
    "이동 편리",
    "식사시간 시청",
    "자기전 시청"
  ],
  "sentiment": {
    "이동 편리": "긍정",
    "식사

In [11]:
# 응답들을 저장할 리스트
content_list = []

total_iterations = len(text_apply)

with tqdm(total=total_iterations) as pbar:
    for i, text in enumerate(text_apply):

        # (Step 1) 각 raw_text에 대해 issue와 sentiment 추출
        input_raw_text = text
        system_message = {"role": "system", "content": sentiment_instructions}
        user_message = {"role": "user", "content": f"{sentiment_few_shots}\n<raw_text>\n{input_raw_text}\n</raw_text>"}
        messages = [system_message, user_message]

        # GPT-4로부터 응답 받기
        content = get_response(messages)
        if not content:
            pbar.update(1)
            time.sleep(1)
            continue
        
        # 응답을 리스트에 저장
        content_list.append((content, input_raw_text))
        
        pbar.update(1)
        time.sleep(1)

100%|██████████| 1000/1000 [47:21<00:00,  2.84s/it]


### 답변 dataframe 변환

In [12]:
# 답변 변환

data_list = []

for content, original_raw_text in content_list:
    parsed_content = convert_to_dict(content)

    raw_text = extract_raw_text(parsed_content)
    if raw_text == "": # raw_text가 비어있는 경우 원본 텍스트로 대체
        raw_text = original_raw_text
    issues = extract_issues(parsed_content) if parsed_content else []
    sentiments = extract_sentiments(parsed_content) if parsed_content else {}
    sentiment_all = extract_sentiment_all(parsed_content) if parsed_content else ""

    for issue in issues:
        sentiment = sentiments.get(issue, 'Unknown')
        data_entry = {
            "raw_text": raw_text,
            "issue": issue,
            "sentiment": sentiment,
            "sentiment_all": sentiment_all
        }
        data_list.append(data_entry)

result_s1 = pd.DataFrame(data_list)
result_s1

,raw_text,issue,sentiment,sentiment_all
0,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정
1,너무 만족하며 사용중입니다,사용 만족,긍정,긍정
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정
3,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정
4,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정
...,...,...,...,...
2476,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,중립,긍정
2477,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정
2478,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정
2479,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정


In [13]:
# 결과 중간 저장
result_s1.to_pickle('result/제공_네이버쇼핑리뷰_S1_감성분석.pkl')
result_s1.to_excel('result/제공_네이버쇼핑리뷰_S1_감성분석.xlsx', index=False)

## Step 2) theme 생성(군집분석)

In [14]:
# issue에서 중복 제거
unique_issues = result_s1['issue'].drop_duplicates().tolist()

In [15]:
# 이슈 갯수
len(unique_issues)

1157

In [16]:
print(theme_instructions)

너는 제품 리뷰 키워드를 군집화하고 분류하는 전문가야. 다음 지시사항에 따라 결과를 생성해줘.
<지시사항>
1. 이슈들을 의미적으로 비슷한 그룹으로 나누고, 각 그룹의 공통 테마(theme)를 도출해줘
2. 답변은 코드블록이나 마크다운 포맷 없이, 순수한 JSON 포맷으로 제공한다. ```json 같은 형식은 사용하지 않는다.
</지시사항>



In [17]:
print(theme_few_shots)

{
  "화질 만족": ["화질 우수", "좋은 화질", "선명한 화질"],
  "사용 편의성": ["리모컨 사용 편리", "메뉴 구성 직관적"],
  "배송 서비스": ["빠른 배송", "친절한 배송"]
}



### Batch 100

In [20]:
# 유니크한 이슈들을 GPT API로 분류
themes = []
batch_size = 100  # 한번에 너무 많은 데이터를 보내지 않기 위해 배치 처리
for i in range(0, len(unique_issues), batch_size):
    batch_issues = unique_issues[i:i+batch_size]
    themes.append(classify_issues(', '.join(batch_issues)))

os.system('say "작업이 완료되었습니다."')
# # 배치 결과를 합쳐서 하나의 테마 리스트로 변환
# theme_list = [theme.strip() for theme in ', '.join(themes).split(',')]

0

In [21]:
themes

['{\n  "제품 품질 및 성능": ["좋은 제품", "품질 만족", "우수한 사운드", "우수한 화질", "좋은 화질", "짱짱한 사운드", "좋은 소리", "튼튼한 기둥", "내구성", "만족스러운 화질", "좋은 TV", "상품 만족", "만족스러운 소비", "제품 만족"],\n  "디자인 및 외관": ["디자인 만족", "예쁜 디자인", "이쁜 디자인", "훌륭한 디자인", "깔끔한 디자인", "뒷면 색상"],\n  "가격 및 가치": ["저렴한 가격", "좋은 가격", "많은 할인", "가격 대비 만족", "가격 만족"],\n  "설치 및 배송 서비스": ["빠른 설치 배송", "설치 서비스 만족", "직접 배송 및 설치", "친절한 설치", "친절한 설치 서비스", "친절한 설치기사", "섬세한 안전 설치", "안전한 배송", "설치 만족", "배송 및 설치 만족", "신속한 설치", "빠른 설치 안내", "친절한 서비스", "친절한 기사", "친절한 배송", "배송 만족", "설명 만족", "배송 시간 지정 가능"],\n  "사용 편의성 및 이동성": ["이동 편리", "무겁지만 이동 용이", "편리한 터치", "터치 작동", "방에서 사용 편리", "편리한 이동", "무선 이동 편리", "이동성", "무선", "편리한 사용", "이동식 티비 만족"],\n  "OTT 및 콘텐츠 시청": ["더 큰 화면으로 OTT 시청 좋음", "OTT 시청", "OTT/유튜브 시청 좋음", "OTT 시청 편리", "넷플릭스 시청 좋음", "TV 시청 가능"],\n  "사용자 만족 및 추천": ["사용 만족", "여자친구 만족", "대만족", "적극 추천", "추천", "추천 의사", "만족", "만족스러운 구매", "구매 만족", "잘 사용 중", "만족스러운 소비", "강추"],\n  "특정 상황 및 용도": ["생일 선물로 좋음", "선물로 좋음", "부모님 선물 적합", "TV 미시청자 추천", "운동 중 사용 좋음", "아이들 교육에 도움", "좁은 공간

In [22]:
# 각 theme의 key를 추출하여 총 길이를 계산
total_keys_length = sum(len(json.loads(theme).keys()) for theme in themes)
total_keys_length

112

In [25]:
merged_themes = merge_themes(themes)

In [26]:
result_s2 = result_s1.copy()

In [27]:
# 'theme' 컬럼을 생성
result_s2['theme'] = result_s2['issue'].apply(lambda x: find_theme(merged_themes, x))
result_s2

,raw_text,issue,sentiment,sentiment_all,theme
0,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정,제품 품질 및 성능
1,너무 만족하며 사용중입니다,사용 만족,긍정,긍정,사용자 만족 및 추천
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정,설치 및 배송 서비스
3,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정,설치 및 배송 서비스
4,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정,사용자 만족 및 추천
...,...,...,...,...,...
2476,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,중립,긍정,캠핑 및 야외 활용
2477,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정,기대 및 만족도
2478,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정,화질 및 화면
2479,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정,사용 편의성 및 이동성


### tid 생성

In [28]:
# "raw_text" 값에 따른 고유 번호 생성 (0부터 시작).
unique_ids, _ = pd.factorize(result_s2['raw_text']) # 인덱스만 사용하므로 두 번째 반환값은 무시

# "tid" 값 생성 (0001, 0002 형태로 고유 번호 할당)
result_s2['tid'] = ['text' + str(i+1).zfill(4) for i in unique_ids]

# "tid" 컬럼을 "raw_text" 앞에 삽입
result_s2 = result_s2[['tid'] + [col for col in result_s2.columns if col != 'tid']]
result_s2

,tid,raw_text,issue,sentiment,sentiment_all,theme
0,text0001,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정,제품 품질 및 성능
1,text0002,너무 만족하며 사용중입니다,사용 만족,긍정,긍정,사용자 만족 및 추천
2,text0003,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정,설치 및 배송 서비스
3,text0003,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정,설치 및 배송 서비스
4,text0004,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정,사용자 만족 및 추천
...,...,...,...,...,...,...
2476,text0991,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,중립,긍정,캠핑 및 야외 활용
2477,text0991,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정,기대 및 만족도
2478,text0992,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정,화질 및 화면
2479,text0992,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정,사용 편의성 및 이동성


In [30]:
result_s2.to_pickle('result/제공_네이버쇼핑리뷰_S2_감성테마.pkl')
result_s2.to_excel('result/제공_네이버쇼핑리뷰_S2_감성테마.xlsx', index = False)